In [86]:
import pandas as pd
import numpy as np

### Loading CSV into pandas dataframe for data cleaning

In [87]:
suicides = pd.read_csv('suicides.csv')

#### Initial look at the data for data cleaning

In [88]:
suicides.head()

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,EN,2021-07-06T04:00:00.000Z
1,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,EN,2021-07-06T04:00:00.000Z
2,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,EN,2021-07-06T04:00:00.000Z
3,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,EN,2021-07-06T04:00:00.000Z
4,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,EN,2021-07-06T04:00:00.000Z


#### Remove metadata columns and missing data to create a usable dataset for EDA

In [89]:
# Check columns to identify which columns to drop that contain missing data over 60%
suicides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4392 entries, 0 to 4391
Data columns (total 34 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   IndicatorCode               4392 non-null   object 
 1   Indicator                   4392 non-null   object 
 2   ValueType                   4392 non-null   object 
 3   ParentLocationCode          4392 non-null   object 
 4   ParentLocation              4392 non-null   object 
 5   Location type               4392 non-null   object 
 6   SpatialDimValueCode         4392 non-null   object 
 7   Location                    4392 non-null   object 
 8   Period type                 4392 non-null   object 
 9   Period                      4392 non-null   int64  
 10  IsLatestYear                4392 non-null   bool   
 11  Dim1 type                   4392 non-null   object 
 12  Dim1                        4392 non-null   object 
 13  Dim1ValueCode               4392 

In [90]:
# Drop missing data columns to clean the dataset
suicides.dropna(axis='columns', how='all', inplace=True)
suicides.head()

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,Dim1 type,Dim1,Dim1ValueCode,Dim2 type,Dim2,Dim2ValueCode,FactValueNumeric,Value,Language,DateModified
0,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,Sex,Both sexes,BTSX,Age Group,15-24 years,YEARS15-24,0.0,0.0,EN,2021-07-06T04:00:00.000Z
1,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,Sex,Both sexes,BTSX,Age Group,25-34 years of age,YEARS25-34,0.0,0.0,EN,2021-07-06T04:00:00.000Z
2,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,Sex,Both sexes,BTSX,Age Group,35-44 years of age,YEARS35-44,0.0,0.0,EN,2021-07-06T04:00:00.000Z
3,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,Sex,Both sexes,BTSX,Age Group,45-54 years,YEARS45-54,0.0,0.0,EN,2021-07-06T04:00:00.000Z
4,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,Sex,Both sexes,BTSX,Age Group,65-74 years,YEARS65-74,0.0,0.0,EN,2021-07-06T04:00:00.000Z


In [91]:
#Check values of Dim2ValueCode to determine how to split the strings
suicides.Dim2ValueCode.value_counts()

YEARS15-24     549
YEARS25-34     549
YEARS35-44     549
YEARS45-54     549
YEARS65-74     549
YEARS75-84     549
YEARS85PLUS    549
YEARS55-64     549
Name: Dim2ValueCode, dtype: int64

In [92]:
#Replace YEARS with "" to prep for a column split
suicides['Dim2ValueCode'] = suicides['Dim2ValueCode'].replace('YEARS', '', regex=True)
print(suicides['Dim2ValueCode'].head())

#Split Dim2ValueCode (Age) into 2 columns age min/max for EDA and queries
suicides['AgeMin'] = suicides['Dim2ValueCode'].str.split('-', expand=True)[0]
suicides['AgeMax'] = suicides['Dim2ValueCode'].str.split('-', expand=True)[1]

#Check dataframe for accurate split and new columns
suicides.head()

0    15-24
1    25-34
2    35-44
3    45-54
4    65-74
Name: Dim2ValueCode, dtype: object


,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,Dim1ValueCode,Dim2 type,Dim2,Dim2ValueCode,FactValueNumeric,Value,Language,DateModified,AgeMin,AgeMax
0,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,BTSX,Age Group,15-24 years,15-24,0.0,0.0,EN,2021-07-06T04:00:00.000Z,15,24
1,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,BTSX,Age Group,25-34 years of age,25-34,0.0,0.0,EN,2021-07-06T04:00:00.000Z,25,34
2,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,BTSX,Age Group,35-44 years of age,35-44,0.0,0.0,EN,2021-07-06T04:00:00.000Z,35,44
3,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,BTSX,Age Group,45-54 years,45-54,0.0,0.0,EN,2021-07-06T04:00:00.000Z,45,54
4,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,BTSX,Age Group,65-74 years,65-74,0.0,0.0,EN,2021-07-06T04:00:00.000Z,65,74


In [93]:
# Identify columns to remove to create a clean dataset for EDA
suicide_columns = suicides.columns.to_list()
print(suicide_columns)

#Add desired columns to a list
desired_columns = ['ParentLocation', 'Location', 'Period', 'Value', 'Dim1', 'AgeMin', 'AgeMax']

['IndicatorCode', 'Indicator', 'ValueType', 'ParentLocationCode', 'ParentLocation', 'Location type', 'SpatialDimValueCode', 'Location', 'Period type', 'Period', 'IsLatestYear', 'Dim1 type', 'Dim1', 'Dim1ValueCode', 'Dim2 type', 'Dim2', 'Dim2ValueCode', 'FactValueNumeric', 'Value', 'Language', 'DateModified', 'AgeMin', 'AgeMax']


### Chosing columns for EDA (Metadata)

-- Metadata on columns for Crude Suicide Rates (per 100 000 population)-- <br>
ParentLocation = Region ('Americas', 'Western Pacific', 'Europe', 'Africa', 'South-East Asia', 'Eastern Mediterranean') <br>
Location = Country data surveyed (Multitude) <br>
Period = Year data collected <br>
Value = Crude Suicide Rates (per 100 000 population) <br>
Sex = Gender (Male/Female/Both) <br>
AgeMin / AgeMax = Age ranges for data (Assume 100 yrs old is max)

In [94]:
suicides_clean = suicides[desired_columns].rename(columns={'Dim1': 'Sex'})
suicides_clean.head()

,ParentLocation,Location,Period,Value,Sex,AgeMin,AgeMax
0,Americas,Antigua and Barbuda,2019,0.0,Both sexes,15,24
1,Americas,Antigua and Barbuda,2019,0.0,Both sexes,25,34
2,Americas,Antigua and Barbuda,2019,0.0,Both sexes,35,44
3,Americas,Antigua and Barbuda,2019,0.0,Both sexes,45,54
4,Americas,Antigua and Barbuda,2019,0.0,Both sexes,65,74


In [95]:
# Adjust Sex column where Both sexes = Both for simplicity
suicides_clean['Sex'] = suicides_clean.Sex.apply(lambda sex: 'Both' if sex == 'Both sexes' else sex)

# Check for correct changes
suicides_clean.Sex.value_counts()

Both      1464
Female    1464
Male      1464
Name: Sex, dtype: int64

### Check/adjust data types to complete data cleaning

In [96]:
suicides_clean.dtypes

ParentLocation     object
Location           object
Period              int64
Value             float64
Sex                object
AgeMin             object
AgeMax             object
dtype: object

#### Convert AgeMin/Max to int datatype for EDA

In [97]:
#Check values of both AgeMin/Max columns to see to check for strings to remove
suicides_clean.AgeMin.value_counts()
suicides_clean[suicides_clean.AgeMin == '85PLUS'].head()

,ParentLocation,Location,Period,Value,Sex,AgeMin,AgeMax
6,Americas,Antigua and Barbuda,2019,0.0,Both,85PLUS,None
13,Americas,Antigua and Barbuda,2019,0.0,Female,85PLUS,None
21,Americas,Antigua and Barbuda,2019,0.0,Male,85PLUS,None
26,Americas,Barbados,2019,0.0,Both,85PLUS,None
33,Americas,Barbados,2019,0.0,Female,85PLUS,None


In [98]:
#Convert strings to int to prep for dtype change
suicides_clean['AgeMin'] = suicides_clean.AgeMin.apply(lambda age: 85 if age == '85PLUS' else age)

#Check for correct changes
suicides_clean.AgeMin.value_counts()

15    549
25    549
35    549
45    549
65    549
75    549
85    549
55    549
Name: AgeMin, dtype: int64

In [99]:
#Check for values of AgeMax to clean
suicides_clean.AgeMax.unique()

array(['24', '34', '44', '54', '74', '84', None, '64'], dtype=object)

In [100]:
suicides_clean.isnull().sum()

ParentLocation      0
Location            0
Period              0
Value               0
Sex                 0
AgeMin              0
AgeMax            549
dtype: int64

In [101]:
#Fill na to prep for conversion to int dtype
#Assume 100 is the max
suicides_clean.fillna(value={'AgeMax': 100}, inplace=True)

#Check for correct changes
suicides_clean.AgeMax.unique()

array(['24', '34', '44', '54', '74', '84', 100, '64'], dtype=object)

In [102]:
#Convert to int dtype for EDA and to complete data cleaning
suicides_clean['AgeMin'] = pd.to_numeric(suicides_clean.AgeMin)
suicides_clean['AgeMax'] = pd.to_numeric(suicides_clean.AgeMax)

#Check for correct changes to complete data cleaning
suicides_clean.dtypes

ParentLocation     object
Location           object
Period              int64
Value             float64
Sex                object
AgeMin              int64
AgeMax              int64
dtype: object

### Univariate Data Profiling

Initial summary statistics to get a general picture of the data. <br>
Also to generate questions to answer using the data and potential model building